In [ ]:
import numpy as np
import torch, torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Neural Networks using PyTorch framework
---
![](resources/torch.png)
## Comparing performance of classical networks to convolutional neural networks 

## Typical Deep Learning workflow:

1. Load your training datasets, and(if needed) convert them into PyTorch datasets
2. Build PyTorch-DataLoaders using your datasets, set shuffle = True and define batch size
3. Define the neural network structure
4. Training process:
    - Define optimizer
    - Define loss function
    - Define # of training iterations
    - Train your model
5. Evaluation process:
    - Use your model to predict labels for your test set
    - evaluate accuracy with true labels

# Part I: Pair programming

## Get the training data

In [ ]:
# Get the data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

training_data = torchvision.datasets.MNIST('data/', train=True, download=True, transform=transform)
testing_data = torchvision.datasets.MNIST('data/', train=False, download=True, transform=transform)

In [ ]:
training_data

In [ ]:
testing_data

## Look at the data more closely

In [ ]:
test_data_point = training_data.data[0]
test_target = training_data.targets[0]

In [ ]:
print(f'Shape of data sample: {np.array(test_data_point).shape}')
print(f'First row of data sample: {np.array(test_data_point)[0]}')

In [ ]:
fig = plt.figure(figsize=(10,10))

plt.imshow(test_data_point)
plt.show()

print(f'Label: {test_target}')

## Create dataloaders to feed data into our neural network

In [ ]:
batch_size = 16
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testing_data, batch_size=batch_size, shuffle=True)

# Define the network

In [ ]:
class NeuralNet(torch.nn.Module):
    
    def __init__(self, input_dim, num_classes):
        super(NeuralNet, self).__init__()
        
        self.input_dim = input_dim
        self.num_classes = num_classes
        
        self.linear_layer1 = torch.nn.Linear(self.input_dim, 100)
        self.linear_layer2 = torch.nn.Linear(100, 50)
        self.linear_layer3 = torch.nn.Linear(50, self.num_classes)
        
    def forward(self, x):
        
        # Layer 1 + activation
        x = self.linear_layer1(x.view(-1, self.input_dim))
        x = F.sigmoid(x)
        
        # Layer 2 + activation
        x = self.linear_layer2(x)
        x = F.sigmoid(x)
        
        # Layer 3 + activation
        x = self.linear_layer3(x)
        x = F.sigmoid(x)
        
        return x

## Training loop

In [ ]:
# Build an object of the neural network
# Optimizer
# Define a Loss Function
# Run the training loop

neural_net = NeuralNet(784, 10)
optimizer = torch.optim.SGD(params=neural_net.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# training loop:

for epoch in range(10):
    running_loss = 0.0
    
    for i, (x, y) in enumerate(train_loader, 1):
        
        # set optimizer gradients to zero
        optimizer.zero_grad()
        
        # forward pass
        predictions = neural_net.forward(x)
        
        # backward pass + optimization step
        loss = loss_fn(predictions, y)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        
        if i % 1000 == 0:
            print(f'Epoch: {epoch}, loss: {running_loss / i}')
        
    print(f'Loss after epoch: {epoch} = {running_loss / len(train_loader)}')

# Evaluation loop

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = neural_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

## Compare regular Multilayer-perceptron performance against Convolutional neural network

### How to compute output size after convolutional layer ??:
- If you just set the channel input size, channel output size, kernel size for your Conv2D function -> output width = input_width - kernel_width + 1
- Same for output height

### How to compute output size after pooling layer ??:
- If you just set the kernel size of your pooling layer, without inputing any other arguments -> output width = input_width / kernel_width
- Same for output height

#### If you change other input arguments to the Conv/MaxPool functions, the output sizes will be computed as explained in the docs:
https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d

- Nice visualizations of different kernel/filter/convolution strategies: https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md

In [ ]:
class ConvNet(torch.nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.pool = torch.nn.MaxPool2d(2, 2)
        
        self.fc1 = torch.nn.Linear(16 * 4 * 4, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        
        # Input(batch_size, 1, 28, 28)
        x = F.relu(self.conv1(x))
        
        # Input(batch_size, 6, 24, 24)
        x = self.pool(x)
        
        # Input(batch_size, 16, 12, 12)
        x = F.relu(self.conv2(x))
        
        # Input(batch_size, 16, 8, 8)
        x = self.pool()
        
        # Input(batch_size, 16, 4, 4)
        x = x.view(-1, 16 * 4 * 4)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [ ]:
# Build an object of the neural network
# Optimizer
# Define a Loss Function
# Run the training loop

neural_net = ConvNet()
optimizer = torch.optim.SGD(params=neural_net.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# training loop:
for epoch in range(10):
    
    running_loss = 0.0
    
    for i, (x, y) in enumerate(train_loader, 1):
        
        # set optimizer gradients to zero
        optimizer.zero_grad()
        
        # forward pass
        predictions = neural_net.forward(x)
        
        # backward pass + optimization step
        loss = loss_fn(predictions, y)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        
        if i % 1000 == 0:
            print(f'Epoch: {epoch}, loss: {running_loss / i}')
        
    print(f'\nLoss after epoch: {epoch} = {running_loss / len(train_loader)}')

# Evaluation loop

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = neural_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

# Part II: Build your own Neural Network classifiers:

### Todos:
1. Load the CIFAR 10 train and test dataset from the torchvision library that we have used above for the MNIST data:
Documentation: https://pytorch.org/docs/stable/torchvision/datasets.html

2. Create DataLoaders for the training and test size:
    - experiment with different batch sizes
3. Create one fully connected model and another Convolutional Neural Network, for each experiment with different layer sizes(# of neurons) and layer types:
    - Conv layers preprocess the data
    - Pooling layers preprocess the data
    - Fully connected layer need to be added at the end to classify the data

4. Evaluate the prediction accuracy(all correct classified points / number of points) of your Fully-connected and Convolutional Neural Networks

5. Evaluate prediction accuracy of each class, e.g.: Correctly classified: 60% of planes, 70% of cars, 30% of housed etc.